In [1]:
import os                                   
import numpy as np                                     
from scipy import optimize, stats                      
import pandas as pd
import time

In [2]:
# for i in range(16):

#path for main direc
path = "/Users/nicoledawney/Desktop/imaris_statistics/"
#address of desired end location
output_path = "/Users/nicoledawney/Desktop/ALAn_input"
folder = os.listdir(path) #returns list of folders in directory

In [3]:
folder

["Etoposide_48hrs_fixedat0_b12_2022-07-26_Nicole's protocol_08.37.11_Admin_Statistics",
 "Etoposide_48hrs_fixedat0_b4_2022-07-26_Nicole's protocol_08.24.44_Admin_Statistics",
 "Etoposide_48hrs_24hrsEtop_b10_2022-07-25_Nicole's protocol_16.26.39_Admin_Statistics",
 "Etoposide_48hrs_fixedat0_b13_2022-07-26_Nicole's protocol_08.38.55_Admin_Statistics",
 '.DS_Store',
 "Etoposide_48hrs_fixedat0_b9_2022-07-26_Nicole's protocol_08.32.34_Admin_Statistics",
 "Etoposide_48hrs_fixedat0_b6_2022-07-26_Nicole's protocol_08.27.56_Admin_Statistics",
 "Etoposide_48hrs_24hrsEtop_b2_2022-07-25_Nicole's protocol_16.13.08_Admin_Statistics",
 "Etoposide_48hrs_24hrsEtop_b12_2022-07-25_Nicole's protocol_16.29.54_Admin_Statistics",
 "Etoposide_48hrs_fixedat0_b3_2022-07-26_Nicole's protocol_08.23.00_Admin_Statistics",
 "Etoposide_48hrs_24hrsEtop_b3_2022-07-25_Nicole's protocol_16.14.47_Admin_Statistics",
 "Etoposide_48hrs_24hrsEtop_b4_2022-07-25_Nicole's protocol_16.16.23_Admin_Statistics",
 "Etoposide_48hrs_24

In [4]:
def subfolder_name_lists(folder_address, list_of_folders, output_path):
    
    labels = []
    list_of_folders.sort()
    list_of_folders = [i for i in list_of_folders if i != '.DS_Store'] #remove unwanted element in list
#     print(list_of_folders)
    for item in list_of_folders:
        x = item.split('_')
#         print(x)
        label = x[0] + x[1] +x[2] +x[3] #take only the desired labels for the collated file
                                    #in this case: e.g. "100k + 1hr + a1"
#         print(str(label))
        labels.append(str(label)) #append each label to a list for later use
#     print(labels)
    
    positions = [] #list of position files names
    volumes = [] #list of volume file names
    subfolder_addresses = [] #list of subfolder paths
    for item in list_of_folders:
#         item = list_of_folders[i+1]
        subfolder = folder_address + "/" + item  #subfolder address
        subfolder_addresses.append(subfolder)
        files = os.listdir(subfolder) #returns list of files in subfolder
        #next two lines will assign position and volume csv file names
        position = [f for f in files if f[-12:] == "Position.csv"]
        volume = [f for f in files if f[-10:] == "Volume.csv"]
        #append position/volume file names into their respective lists
        positions.append(position)
        volumes.append(volume)
#         print(position,volume)
    #next two lines transform the lists of lists of strings into lists of strings for ease of use later
    positions = [''.join(i) for i in positions]
    volumes = [''.join(i) for i in volumes]
    
    #need to go into folder then subfolder then pd.read_excel the desired csv, then append to a list of dfs
    #use list of subfolder addresses to bypass first step, then just say for each subfolder address add on our
    #position of volume files suffix and pd.read_excel that sucker
    position_dfs = []
    volume_dfs = []
    
    for i in range(len(subfolder_addresses)):
        
        #read in position csv's as data frames and append to list
        position_address = subfolder_addresses[i] + "/" + positions[i]
        df = pd.read_csv(position_address, encoding = "utf-8", skiprows = 3)
        df = df.drop(['Unit', "Category", "Collection", "Time", "CellID", "ID", "Unnamed: 9"], axis = 1) #dropping unwanted columns
        position_dfs.append(df)
        
        #read in volume csv's as data frames and append to list
        volume_address = subfolder_addresses[i] + "/" + volumes[i]
        volume_dfs.append(pd.read_csv(volume_address, encoding = "utf-8", skiprows = 3))
    
    #taking the one column we want from the volume csv's and merging it with the processed position csv's
    merged_dfs = []
    for i in range(len(volume_dfs)):
        df = volume_dfs[i]
        sub_df = df["Nucleus Volume"] #single column we want
        p_df = position_dfs[i]
        merged_df = pd.concat([p_df, sub_df], axis=1)
        merged_dfs.append(merged_df)
        #saves collated data as csv's in desired location
        merged_df.to_csv(output_path + '/' + labels[i] + '.csv', index = False)
        
    return merged_dfs, labels

In [5]:
toc = time.time()
subfolder_name_lists(path, folder, output_path)
tic = time.time()
print(tic-toc)

0.39849090576171875
